### Problem statement

You are appointed as SA for **AdventureWork Bike**. It is company which is into selling sport bikes. The Company wants to 
explore Snowpark and wants to make their a analytst and Data scientist fimilar with snowpark. As the SA for the AdventureWork
you want to help the team to explore the capabilities of snowpark for data cleansing and for creating models.


The Analyst have provided you with two csv files containing the customer data and Sales.

**Prerequisite** :-  
. Knowledge of jupyter notebook   
. Knowlegde of Pandas , Numpy and similar python Packages  
. Snowflake account and previlege to create Databases in Snowflake environment 

In [1]:
#import the snowflake libraries
from snowflake.snowpark.session import Session
import getpass

In [2]:
#Authentication

user = input("Username:")
pwd = getpass.getpass("Password for " + user + ":")
wh = input("Warehouse:") # or create a new warehouse an runtime
role = input("Role:")

connection_parameters = {
  "account": "sfpscogs-rp022az",
  "user": user,
  "password": pwd,
  "role": role,
  "warehouse": wh}

session = Session.builder.configs(connection_parameters).create()

Username:rpegu
Password for rpegu:········
Warehouse:demo_wh
Role:demo_role


In [83]:
# test the connection to snowflake is working
c_date = session.sql('select current_date()').collect()

In [4]:
# create database in snowflake environment

# Create Databse
session.sql("CREATE OR REPLACE DATABASE Adventurework_DB").collect()
session.sql("CREATE or replace schema RAW").collect()

[Row(status='Schema RAW successfully created.')]

In [5]:
## will use the pandas library to load the data
import pandas as pd

# Loading from local CSV-files
AWCustomer_df = pd.read_csv('Data/AWCustomers.csv')

AWSales_df = pd.read_csv('Data/AWSales.csv')


In [7]:
## Pandas Dataframes
AWCustomer_df.head(5)

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated
0,21173,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,...,Bachelors,Clerical,M,M,1,3,0,1,81916,2017-03-06
1,13249,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,...,Partial College,Clerical,M,M,1,2,1,2,81076,2017-03-06
2,29350,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,...,Bachelors,Clerical,F,S,0,3,0,0,86387,2017-03-06
3,13503,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,...,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2017-03-06
4,22803,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,...,Partial College,Skilled Manual,M,S,1,1,0,0,51804,2017-03-06


In [23]:

AWSales_df.head(5)

,CustomerID,BikeBuyer,AvgMonthSpend
0,21173,1,50.97
1,13249,1,53.11
2,29350,1,54.08
3,13503,1,56.93
4,22803,1,55.41


In [22]:
print(AWCustomer_df.shape)
print(AWSales_df.shape)

(18361, 24)
(18355, 3)


In [7]:
# Upload to Snowflake

#write the dataframe in AW_Customer in 
session.write_pandas(AWCustomer_df, table_name="AW_CUSTOMERS", auto_create_table=True, overwrite=True)

session.write_pandas(AWSales_df, table_name='AW_SALES', auto_create_table=True, overwrite=True)

Its a good time to see whats happening at the snowflake site. 


![snf_worksheet_1](images/snf_ws_1.png)

Now lets check the Query History tab in snowflake snowsight ui.


![Query History](Images/snf_qh_1.png)

Lets create a snowpark Dataframe and compare it with Pandas dataframe.

In [9]:
# creating snowpark DataFrame

snowpark_df = session.table('RAW.AW_CUSTOMERS')

As we know often Analyst and Data Scientist faces memory issues when working with jupyter notebook or similar tools. lets check 
the memory size for dataframes.

In [10]:
import numpy as np
import  sys

print('Size in MB of Snowpark DataFrame in Memory:\n', np.round(sys.getsizeof(snowpark_df) / (1024.0**2), 2))

print('Size in MB of pandas DataFrame in Memory:\n', np.round(sys.getsizeof(AWCustomer_df) / (1024.0**2), 2))

Size in MB of Snowpark DataFrame in Memory:
 0.0
Size in MB of pandas DataFrame in Memory:
 19.55


Display the data in snowpark dataframe

In [11]:
# to  display the data 
snowpark_df.show(5)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CustomerID"  |"Title"  |"FirstName"  |"MiddleName"  |"LastName"  |"Suffix"  |"AddressLine1"          |"AddressLine2"  |"City"       |"StateProvinceName"  |"CountryRegionName"  |"PostalCode"  |"PhoneNumber"        |"BirthDate"  |"Education"      |"Occupation"    |"Gender"  |"MaritalStatus"  |"HomeOwnerFlag"  |"NumberCarsOwned"  |"NumberChildrenAtHome"  |"TotalChildren"  |"YearlyIncome"  |"LastUpdated"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
We have already seen the size of snowpark memory is 0. It will be good idea, at this time to check 
what happening in snowflake side. So let open your snowflake Query history tab and check for more info.

**Query History**



![snf_QH_2](images/snf_qh_2.png)

It doesn't look very pretty !!. let convert into pandas and run the code.

In [13]:
snowpark_df.limit(5).to_pandas()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated
0,21173,None,Chad,C,Yuan,None,7090 C. Mount Hood,None,Wollongong,New South Wales,...,Bachelors,Clerical,M,M,1,3,0,1,81916,2017-03-06
1,13249,None,Ryan,None,Perry,None,3651 Willow Lake Rd,None,Shawnee,British Columbia,...,Partial College,Clerical,M,M,1,2,1,2,81076,2017-03-06
2,29350,None,Julia,None,Thompson,None,1774 Tice Valley Blvd.,None,West Covina,California,...,Bachelors,Clerical,F,S,0,3,0,0,86387,2017-03-06
3,13503,None,Theodore,None,Gomez,None,2103 Baldwin Dr,None,Liverpool,England,...,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2017-03-06
4,22803,None,Marshall,J,Shan,None,Am Gallberg 234,None,Werne,Nordrhein-Westfalen,...,Partial College,Skilled Manual,M,S,1,1,0,0,51804,2017-03-06


In [14]:
snowpark_df.columns

['"CustomerID"',
 '"Title"',
 '"FirstName"',
 '"MiddleName"',
 '"LastName"',
 '"Suffix"',
 '"AddressLine1"',
 '"AddressLine2"',
 '"City"',
 '"StateProvinceName"',
 '"CountryRegionName"',
 '"PostalCode"',
 '"PhoneNumber"',
 '"BirthDate"',
 '"Education"',
 '"Occupation"',
 '"Gender"',
 '"MaritalStatus"',
 '"HomeOwnerFlag"',
 '"NumberCarsOwned"',
 '"NumberChildrenAtHome"',
 '"TotalChildren"',
 '"YearlyIncome"',
 '"LastUpdated"']

In [12]:
snowpark_df[['"CustomerID"','"FirstName"','"MiddleName"']].limit(5).to_pandas()

,CustomerID,FirstName,MiddleName
0,21173,Chad,C
1,13249,Ryan,None
2,29350,Julia,None
3,13503,Theodore,None
4,22803,Marshall,J


In [13]:
## Uses the QueryResult - cache in snowflake.

snowpark_df[['"CustomerID"','"FirstName"','"MiddleName"']].limit(5).to_pandas()

,CustomerID,FirstName,MiddleName
0,21173,Chad,C
1,13249,Ryan,None
2,29350,Julia,None
3,13503,Theodore,None
4,22803,Marshall,J


## TRANSFORMATIONS USING SNOWPARK DATAFRAMES ##

 **Note** - DataFrame is considered lazy because it encapsulates the computation or query required to produce a relational dataset.
The computation is not performed until you call a method that performs an *action* (e.g. collect()).

Lets try our hands in some basic transformations. I refered to this [document](https://docs.snowflake.com/ko/developer-guide/snowpark/reference/python/api/snowflake.snowpark.DataFrame.html) by snowflake



In [19]:
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window

**Example 1** Join()

In [71]:


snw_awcust_df = session.table('RAW.AW_CUSTOMERS')
snw_awsales_df = session.table('RAW.AW_SALES')

snw_aw_custsales_df = snw_awcust_df.join(snw_awsales_df, snw_awcust_df['"CustomerID"'] \
                                         ==snw_awsales_df['"CustomerID"'])\
.select (snw_awcust_df['"CustomerID"'].as_("CUST_ID"), F.col('"BirthDate"'), \
                           F.col( '"Education"'),\
 F.col('"Occupation"'),\
 F.col('"Gender"'),\
 F.col('"MaritalStatus"'),\
 F.col('"HomeOwnerFlag"'),\
 F.col('"NumberCarsOwned"'),\
 F.col('"NumberChildrenAtHome"'),\
 F.col('"TotalChildren"'),\
  F.col('"YearlyIncome"'),F.col('"BikeBuyer"'),F.col('"AvgMonthSpend"'))

In [72]:
snw_aw_custsales_df.limit(5).to_pandas()

,CUST_ID,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,BikeBuyer,AvgMonthSpend
0,21173,1987-11-13,Bachelors,Clerical,M,M,1,3,0,1,81916,1,50.97
1,13249,1972-07-21,Partial College,Clerical,M,M,1,2,1,2,81076,1,53.11
2,29350,1985-11-09,Bachelors,Clerical,F,S,0,3,0,0,86387,1,54.08
3,13503,1977-10-18,Partial College,Skilled Manual,M,M,1,2,1,2,61481,1,56.93
4,22803,1975-02-05,Partial College,Skilled Manual,M,S,1,1,0,0,51804,1,55.41


**Example 2** count()

In [73]:
snw_aw_custsales_df.count()

18361

In [74]:
snw_aw_custsales_df.columns

['CUST_ID',
 '"BirthDate"',
 '"Education"',
 '"Occupation"',
 '"Gender"',
 '"MaritalStatus"',
 '"HomeOwnerFlag"',
 '"NumberCarsOwned"',
 '"NumberChildrenAtHome"',
 '"TotalChildren"',
 '"YearlyIncome"',
 '"BikeBuyer"',
 '"AvgMonthSpend"']

In [ ]:
**case sensitive - F.COL does not work**

In [75]:
snw_aw_custsales_df.select("CUST_ID",'"BirthDate"').show()

---------------------------
|"CUST_ID"  |"BirthDate"  |
---------------------------
|21173      |1987-11-13   |
|13249      |1972-07-21   |
|29350      |1985-11-09   |
|13503      |1977-10-18   |
|22803      |1975-02-05   |
|22092      |1975-02-23   |
|11229      |1971-12-03   |
|24179      |1997-03-07   |
|12127      |1976-12-14   |
|19903      |1976-12-04   |
---------------------------



In [76]:
snw_aw_custsales_df.write.mode("overwrite").save_as_table("AW_CUSTOMER_SALES")

In [48]:
type(snw_aw_custsales_df)

snowflake.snowpark.dataframe.DataFrame

In [84]:
snw_aw_custsales_df.select(F.datediff('year',"BirthDate",c_date )).show()

TypeError: 'DATEDIFF' expected Column or str, got: <class 'list'>